In [ ]:
# import module

import pickle
from core.MV2 import MV2
from core.JIG import JIG
from core.SG2 import SG2 
from core.AWL import AWL

In [ ]:
# GIPS 입력 긴 문자열 -> 리스트 또는 셋으로 변경

def main(payload_path, signature_path, stopword_path, virtual_vector_path, big_group_path,
         window_size, K, M, # MV2 파라미터
         thetaJ,  # JIG 파라미터
         vector_size, eps, minpts, ngram, hh1_size, hh2_size, hh3_size, ratio # SG2, AWL 파라미터
        ):
    
    with open(payload_path, 'rb') as f:
        payloads_data = pickle.load(f)

    print(len(payloads_data))
    payloads = [x for x in payloads_data]

    print(len(payloads))

    # 빅 그룹 식별
    minhashed_virtual_vectors = MV2(payloads=payloads, window_size=window_size, K=K, M=M)
    with open(virtual_vector_path, 'wb') as f:
        pickle.dump(minhashed_virtual_vectors, f)

    big_group_indices = JIG(vectors=minhashed_virtual_vectors, thetaJ=thetaJ)
    with open(big_group_path, 'wb') as f:
        pickle.dump(big_group_indices, f)
    
    big_group_payloads = []
    non_big_group_paylaods = []

    for idx, payload in enumerate(payloads):
        if idx in big_group_indices:
            big_group_payloads.append(payload)
        else:
            non_big_group_paylaods.append(payload)
    
    # 시그니처 생성
    cluster_signatures = SG2(payloads=big_group_payloads, window_size=window_size, vector_size=vector_size, 
                             eps=eps, minpts=minpts, ngram=ngram, hh1_size=hh1_size, hh2_size=hh2_size, hh3_size=hh3_size, ratio=ratio)
    
    with open(signature_path, 'wb') as f:
        pickle.dump(cluster_signatures, f)
    
    stopwords = AWL(payloads=non_big_group_paylaods, ngram=ngram, hh1_size=hh1_size, hh2_size=hh2_size, hh3_size=hh3_size, ratio=ratio)

    with open(stopword_path, 'wb') as f:
        pickle.dump(stopwords, f)

In [ ]:
# 파라미터

file_path = './datasets/pe_string.pkl'
virtual_vector_path = f'./res/PE_virtual_vector.pkl'
big_group_indices_path = f'./res/PE_big_group.pkl'
signature_path = f'./res/PE_signature.pkl'
stopword_path = f'./res/PE_stopword.pkl'

K = 64
M = 2 ** 14
thetaJ = 0.6
window_size = 4
vector_size = 512
eps = 0.4
minpts = 5
ngram = 4
hh1_size = 3000
hh2_size = 3000
hh3_size = 3000
ratio = 0.1

In [8]:
main(payload_path=file_path, signature_path=signature_path, stopword_path=stopword_path,
         virtual_vector_path=virtual_vector_path, big_group_path=big_group_indices_path,
         window_size=window_size, vector_size=vector_size, K=K, M=M, thetaJ=thetaJ,
         eps=eps, minpts=minpts, ngram=ngram, hh1_size=hh2_size, hh2_size=hh2_size, hh3_size=hh3_size, ratio=ratio)
    

29909
29909
make minhashed vector


100%|██████████| 29909/29909 [02:39<00:00, 187.30it/s]


checking big group


100%|██████████| 29909/29909 [02:24<00:00, 207.44it/s]


chunking


100%|██████████| 23402/23402 [00:31<00:00, 746.74it/s]


start DBSCAN
end DBSCAN
make signature


100%|██████████| 3/3 [00:15<00:00,  5.31s/it]


end signature
start whtie list
end white list


In [9]:
import pickle
with open('./res/PE_signature.pkl', 'rb') as f:
    signature = pickle.load(f)
    
with open('./res/PE_stopword.pkl', 'rb') as f:
    stopword = pickle.load(f)
    
with open('./res/PE_big_group.pkl', 'rb') as f:
    big_group = pickle.load(f)

In [10]:
print(len(big_group))

23402


In [11]:
print('SIGNAURE')

for i in signature.values():
    for j in i:
        print(j, end=" ")
    print()
    
print('STOPWORD')
for i in stopword:
    print(i)

SIGNAURE
('Delete', 18574) ('System', 13450) ('GetProcAddress', 12917) ('Exception', 12595) ('user32', 12367) ('ExitProcess', 11489) ('advapi32', 10882) ('LoadLibraryA', 10286) ('CloseHandle', 9786) ('kernel32', 9711) ('ReadFile', 8955) ('!This program cannot be run in DOS mode.', 8745) ('RegCloseKey', 8487) ('WriteFile', 8189) ('VirtualAlloc', 7617) ('GetCurrentProcess', 7578) ('GetLastError', 7391) ('This program must be run under Win32', 7348) ('KERNEL32.DLL', 7323) ('CharNextA', 6898) ('CreateFileA', 6272) ('USER32.dll', 5974) ('GetStdHandle', 5451) ('TObject', 5392) ('GetCurrentThread', 5379) ('KERNEL32.dll', 5162) ('GetVersion', 4806) ('Strings', 3878) ('shell32.dll', 3590) ('RegOpenKeyA', 3474) ('ADVAPI32.dll', 3466) ('Varian', 3187) ('sprintf', 2482) ('`.data', 2466) ('SysFreeString', 2440) ('ivByZero', 2202) ('VirtualProtect', 2104) ('FPUMaskValu', 1911) ('Integer', 1904) ('__vbaExceptHandler', 1799) ('\\Delphi\\RTL', 1723) ('`.rdat', 1605) ('~ExC[)', 1497) ('[[[[[[', 1414) ('

In [12]:
filters = [i[0] for i in stopword if i[1] > 99]
for cluster, sig in signature.items():
    for token in sig:
        if token[1] not in filters:
            print(token)

('Delete', 18574)
('System', 13450)
('GetProcAddress', 12917)
('Exception', 12595)
('user32', 12367)
('ExitProcess', 11489)
('advapi32', 10882)
('LoadLibraryA', 10286)
('CloseHandle', 9786)
('kernel32', 9711)
('ReadFile', 8955)
('!This program cannot be run in DOS mode.', 8745)
('RegCloseKey', 8487)
('WriteFile', 8189)
('VirtualAlloc', 7617)
('GetCurrentProcess', 7578)
('GetLastError', 7391)
('This program must be run under Win32', 7348)
('KERNEL32.DLL', 7323)
('CharNextA', 6898)
('CreateFileA', 6272)
('USER32.dll', 5974)
('GetStdHandle', 5451)
('TObject', 5392)
('GetCurrentThread', 5379)
('KERNEL32.dll', 5162)
('GetVersion', 4806)
('Strings', 3878)
('shell32.dll', 3590)
('RegOpenKeyA', 3474)
('ADVAPI32.dll', 3466)
('Varian', 3187)
('sprintf', 2482)
('`.data', 2466)
('SysFreeString', 2440)
('ivByZero', 2202)
('VirtualProtect', 2104)
('FPUMaskValu', 1911)
('Integer', 1904)
('__vbaExceptHandler', 1799)
('\\Delphi\\RTL', 1723)
('`.rdat', 1605)
('~ExC[)', 1497)
('[[[[[[', 1414)
('GetModule